# Ten Sigma Event Extension
Statistical Consequences of Fat Tails (Page 53): [Link to ebook PDF](https://researchers.one/articles/statistical-consequences-of-fat-tails-real-world-preasymptotics-epistemology-and-applications/5f52699d36a3e45f17ae7e36)

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis as scipy_kurtosis 
from sympy.stats import P, E, variance, std, Die, Normal, StudentT
from sympy import Eq, simplify

In [2]:
Z = Normal('Z', 0, 1) # Declare a Normal random variable with mean 0, std 1
T = StudentT('T', 2)

# Replicate Page 53

We want to find P(Gaussian|Event):

\begin{equation*}
\frac{P(Gaussian)*P(Event|Gaussian)}{\Bigl(1-P(Gaussian)\Bigr)*P(Event|NonGaussian)+P(Gaussian)*P(Event|Gaussian)}
\end{equation*}

In [3]:
p_gaussian_list = [0.5, 0.999, 0.9999, 0.99999, 0.999999, 1] # P(Gaussian) values to check for

p_if_gauss = P(Z>10).evalf()
p_if_nongauss = P(T>10).evalf()

In [4]:
1/p_if_gauss # Should be 1.31x10^23

1.31236127104980e+23

In [5]:
1/p_if_nongauss # # Should be 203

202.995049383621

In [6]:
# Evaluate the equation for each value in p_gaussian_list
p_gauss_if_event_list = []
for p_gauss in p_gaussian_list:
    
    numerator = p_gauss * p_if_gauss
    denominator = (1-p_gauss)*p_if_nongauss+p_gauss*p_if_gauss
    
    p_gauss_if_event = numerator/denominator
    
    p_gauss_if_event_list.append(p_gauss_if_event)

In [7]:
p_gaussian_list

[0.5, 0.999, 0.9999, 0.99999, 0.999999, 1]

In [8]:
p_gauss_if_event_list

[1.54679244093540e-21,
 1.54524564849446e-18,
 1.54663776169147e-17,
 1.54677697301803e-16,
 1.54679089409848e-15,
 1.00000000000000]

In [9]:
# Create DataFrame
d = {'P(Gaussian)':p_gaussian_list, 'P(Gaussian|Event)':p_gauss_if_event_list}
page_53_table = pd.DataFrame(d)

In [10]:
page_53_table

,P(Gaussian),P(Gaussian|Event)
0,0.500000,1.54679244093540e-21
1,0.999000,1.54524564849446e-18
2,0.999900,1.54663776169147e-17
3,0.999990,1.54677697301803e-16
4,0.999999,1.54679089409848e-15
5,1.000000,1.00000000000000


# Extension:

What if you fit a new normal distribution after observing the 10 sigma event. I'm not saying it makes sense but let's see what happens.

Let's suppose the event is from one day in 50 years. Then it's a 1 in 365*50 event.

In [11]:
n = 365*50 # Our dataset holds this many points before the 10 sigma event happens
n

18250

In [12]:
normal_array = np.random.normal(size=n)

df = pd.DataFrame(normal_array, columns = ['normal_sample'])

df.head()

,normal_sample
0,-1.685426
1,-0.614605
2,-0.654180
3,-0.631316
4,-1.486185


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
normal_sample,18250.0,0.003066,1.000099,-3.618038,-0.666777,0.000853,0.676138,3.88181


In [14]:
scipy_kurtosis(df.normal_sample, fisher=False)

2.9903270004741063

So before the tail event:
* std=1
* kurtosis=3
* max observation ~= 4

### Add tail event

In [15]:
new_df = df.append({'normal_sample':10},ignore_index=True)

In [16]:
new_df.describe().T

,count,mean,std,min,25%,50%,75%,max
normal_sample,18251.0,0.003613,1.002805,-3.618038,-0.666752,0.000918,0.676495,10.0


In [17]:
scipy_kurtosis(new_df.normal_sample, fisher=False)

3.4990530638632

After adding the tail event:
* std=1
* kurtosis=3.5
* max=10

Rough conclusion, If we add a 10 sigma event to 50 years of daily gaussian data, ...
* The standard deviation doesn't go up much
* The kurtosis goes up from 3 to 3.5.

Which leaves us with:
* New dataset has mu and sigma of a standard normal, but its kurtosis gives it away as being non-gaussian